In [ ]:
#ln[1] 
from ipyparallel import Client 
rc = Client() 
print('Number of clusters running =', len(rc)) 
print('Client ids are:', rc.ids) 
dview = rc[:] 

In [ ]:
import boto3

s3 = boto3.resource("s3") 
my_bucket = s3.Bucket("giribucketforspark3logs") 
all_keys = [] 
for bucket_obj in my_bucket.objects.all(): 
    all_keys.append(bucket_obj.key) 
print('Total number of json objects =', len(all_keys)) 

In [ ]:
%%px
def test1(keys):
    import json
    import boto3
    from collections import Counter
    
    bucket = 'giribucketforspark3logs'
    artistCounter = {}
    songCounter = {}
    
    s3 = boto3.client('s3')
    
    for key in keys:
        obj = s3.get_object(Bucket=bucket, Key=str(key))
        obj = json.loads(obj['Body'].read())
        
        try:
            # Avoid keeping count for 'None' artist
            if obj['artist']:
                artistCounter[str(obj['artist'])] += 1
        except:
            artistCounter[str(obj['artist'])] = 1
        
        try:
            # Avoid keeping count for 'None' song
            if obj['song']:
                songCounter[str(obj['song'])] += 1
        except:
            songCounter[str(obj['song'])] = 1
            
    return Counter(artistCounter), Counter(songCounter)

In [ ]:
import time
start = time.perf_counter() 
dview.scatter('keys',all_keys) 
#%px print(keys[:5]) 
%px y = [test1(keys)] 
y = dview.gather('y') 
print('Time taken to get all results = {:.4f}s'.format(time.perf_counter() - start))

In [ ]:
 
from collections import Counter
import pandas as pd 
artistCounter = Counter({}) 
songCounter = Counter({}) 
for (artists, songs) in y: 
    artistCounter += artists 
    songCounter += songs 
artistCounter = pd.DataFrame(artistCounter.most_common(10),columns=['Artist Name', 'Count'], index=range(1, 11)) 

songCounter = pd.DataFrame(songCounter.most_common(10),columns=['Song Name', 'Count'], index=range(1, 11))  


In [ ]:
#ln[6] 
pd.set_option('display.max_colwidth', None) 
print('Top 10 Artists are:') 
display(artistCounter) 
print('\n\nTop 10 Songs are:') 
display(songCounter) 